## Twitter Setiment Analysis 

### Part 2 extra: sentiment140 and QWERTY Effect

TODO: Explain scope of this part...

### Load Cleaned Data

In [1]:
import time
import load_data as ld

start = time.perf_counter()

df = ld.run_processes()

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 10.2 second(s)


In [2]:
df.head()

,target,text,tokenized,filtered,stemmed
0,0,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,upset cant update his facebook texting might c...,upset cant updat hi facebook text might cri re...
1,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sav...,i dived many times ball managed save 50 rest g...,i dive mani time ball manag save 50 rest go ou...
2,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy like fire,my whole bodi feel itchi like fire
3,0,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no not behaving all im mad why am i here becau...,no not behav all im mad whi am i here becaus i...
4,0,@Kwesidei not the whole crew,not the whole crew,not whole crew,not whole crew


In [4]:
def calc_rsr(txt):
    """Calculates the ratio of characters in 
    the right-side of the QWERTY keyboard, also
    known as RSR (Right-Side Ratio), given a 
    lower-case text object.
    """
    lside = ['q','w','e','r','t',
             'a','s','d','f','g',
             'z','x','c','v','b']   
    
    rside = ['y','u','i','o','p',
             'h','j','k','l',
             'n','m']
    
    txt = str(txt)
    
    sub_string = [x for x in txt]
    lcount = rcount = 0
    
    for i in sub_string:
        if i in lside:
            lcount += 1
        elif i in rside:
            rcount += 1
        else:
            pass
    
    den = rcount+lcount
    if den != 0:
        return rcount / den
    else:
        return 0
    
def map_rsr(list_):
    map_iterator = map(calc_rsr, list_)
    return list(map_iterator)

In [5]:
start = time.perf_counter()
df['rsr'] = map_rsr(df.loc[:, 'tokenized'])
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 70.85 second(s)


In [6]:
df.head()

,target,text,tokenized,filtered,stemmed,rsr
0,0,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,upset cant update his facebook texting might c...,upset cant updat hi facebook text might cri re...,0.404762
1,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sav...,i dived many times ball managed save 50 rest g...,i dive mani time ball manag save 50 rest go ou...,0.379310
2,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy like fire,my whole bodi feel itchi like fire,0.513514
3,0,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no not behaving all im mad why am i here becau...,no not behav all im mad whi am i here becaus i...,0.424242
4,0,@Kwesidei not the whole crew,not the whole crew,not whole crew,not whole crew,0.400000


In [8]:
df.groupby(['target']).mean()

,rsr
target,
0,0.456939
1,0.462562


The means do not look far apart enough for RSR to be a feature of interest. This does not appear to support the QWERTY Effect. A Student t test of difference of means is warranted but in all likelihood it will fail to reject the null hypothesis that there is no difference in mean RSR between negative and positive Tweets.

In [9]:
# save data for analysis
import os

df_sub = df.loc[:,('target','tokenized','filtered','stemmed','rsr')].copy()
dir_ = os.path.join("..","data","2_clean","sentiment140")
filename = "sentiment140_training.csv.gz"
filepath = os.path.join(dir_, filename)

df_sub.to_csv(filepath, index=False, compression='gzip')